In [ ]:
import gym
import numpy as np

# Create the FrozenLake environment
env = gym.make('FrozenLake-v1')

# Set the parameters
num_episodes = 10000
learning_rate = 0.1
discount_factor = 0.99
epsilon = 0.1

# Initialize the Q-table
state_space = env.observation_space.n
action_space = env.action_space.n
q_table = np.zeros((state_space, action_space))

# Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        if np.random.uniform(0, 1) < epsilon:
            # Explore: Choose a random action
            action = env.action_space.sample()
        else:
            # Exploit: Choose the action with the highest Q-value
            action = np.argmax(q_table[state, :])

        next_state, reward, done, _ = env.step(action)

        # Update the Q-table using the Q-learning update rule
        q_table[state, action] += learning_rate * (
                reward + discount_factor * np.max(q_table[next_state, :]) - q_table[state, action])

        state = next_state

# Print the learned Q-table
print(q_table)

# Close the environment
env.close()


[[0.48130663 0.45538179 0.40459107 0.4269724 ]
 [0.28364326 0.2972368  0.21366365 0.39359471]
 [0.31075468 0.27339823 0.26223331 0.28590036]
 [0.19141684 0.07169671 0.05037005 0.09195343]
 [0.49085176 0.30737112 0.38486896 0.29584442]
 [0.         0.         0.         0.        ]
 [0.18995486 0.18240083 0.12137764 0.18638361]
 [0.         0.         0.         0.        ]
 [0.30108276 0.29479794 0.35622174 0.53969676]
 [0.47245904 0.6311741  0.49260539 0.40020258]
 [0.60595428 0.38741066 0.33555638 0.25839696]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.25874135 0.55277729 0.74046285 0.58866823]
 [0.67387712 0.8852763  0.64569    0.74526304]
 [0.         0.         0.         0.        ]]


In [5]:
import gym
import numpy as np

env = gym.make('FrozenLake-v1')

num_episodes = 10000
learning_rate = 0.1
discount_factor = 0.99
epsilon = 0.1

state_space = env.observation_space.n
action_space = env.action_space.n
q_table = np.zeros((state_space,action_space))

for episode in range(num_episodes):
  state = env.reset()
  done = False

  while not done:
    if np.random.uniform(0,1) < epsilon:
      action =  env.action_space.sample()
    else:
      action = np.argmax(q_table[state,:])

    next_state,reward,done,_ = env.step(action)
    q_table[state,action] += learning_rate*(reward + discount_factor*np.max(q_table[next_state:,]) - q_table[state,action])

    state = next_state

print(q_table)
env.close()

[[0.40465046 0.40465046 0.40465046 0.40465046]
 [0.40465046 0.4046487  0.40464975 0.40465028]
 [0.40465046 0.22401029 0.28617607 0.27872084]
 [0.35270664 0.         0.         0.0099    ]
 [0.40464871 0.40464791 0.40464777 0.40465046]
 [0.         0.         0.         0.        ]
 [0.40460887 0.18675261 0.09991931 0.05814122]
 [0.         0.         0.         0.        ]
 [0.4046504  0.2575399  0.27652061 0.23537028]
 [0.05093936 0.08280999 0.38744432 0.12867103]
 [0.07259798 0.12334422 0.07902425 0.40132052]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.37311322 0.05092412 0.04373919 0.04372226]
 [0.03396116 0.40873783 0.         0.01879119]
 [0.         0.         0.         0.        ]]


In [ ]:
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam

# Create the Atari Breakout environment
env = gym.make('Breakout-v1')

# Set the parameters
num_episodes = 1000
max_steps = 500
epsilon = 1.0
epsilon_decay = 0.99
gamma = 0.99
batch_size = 32

# Define the DQN model
model = Sequential()
model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(84, 84, 4)))
model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(env.action_space.n, activation='linear'))
model.compile(loss='mse', optimizer=Adam())

# DQN algorithm
for episode in range(num_episodes):
    state = env.reset()
    state = preprocess_state(state)  # Preprocess the state
    state = np.stack([state] * 4, axis=2)  # Stack 4 consecutive frames
    total_reward = 0
    done = False

    for step in range(max_steps):
        if np.random.rand() <= epsilon:
            action = env.action_space.sample()
        else:
            q_values = model.predict(np.expand_dims(state, axis=0))
            action = np.argmax(q_values[0])

        next_state, reward, done, _ = env.step(action)
        next_state = preprocess_state(next_state)  # Preprocess the next state
        next_state = np.append(state[:, :, 1:], np.expand_dims(next_state, axis=2), axis=2)

        target = reward
        if not done:
            next_q_values = model.predict(np.expand_dims(next_state, axis=0))
            target = reward + gamma * np.max(next_q_values[0])

        q_values = model.predict(np.expand_dims(state, axis=0))
        q_values[0][action] = target

        model.fit(np.expand_dims(state, axis=0), q_values, verbose=0)

        state = next_state
        total_reward += reward

        if done:
            break

    epsilon *= epsilon_decay

    if episode % 100 == 0:
        print("Episode:", episode, "Total Reward:", total_reward)

# Close the environment
env.close()


In [ ]:
!pip install atari-py==0.2.5

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.2/790.2 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for atari-py: filename=atari_py-0.2.5-cp310-cp310-linux_x86_64.whl size=3118540 sha256=c536f4a6d33729bd3cf568983d4e2383634edd8d05d9d827aaed2c6d0e59edcd
  Stored in directory: /root/.cache/pip/wheels/8a/cf/91/546c450c42ab0252fbffb1ad265ac51903aca4a53fa46dddb8
Successfully built atari-py
